In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import pandas as pd
import os
import splitfolders
import shutil
from sklearn.metrics import classification_report
import numpy as np

In [2]:
image_size = (224, 224) 
batch_size = 32
epochs = 50
num_classes = 2


In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential()
model.add(base_model)  
model.add(layers.Flatten())  
model.add(layers.Dense(256, activation='relu'))  
model.add(layers.Dropout(0.5))  
model.add(layers.Dense(num_classes, activation='softmax')) 


model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

In [4]:
img_directory = 'public/public/'  #image directory where two folders are available each folder representing a class

In [5]:
output_dir = 'output_split/'  #directory where split fodlers will be saved
folders = ['globally_sclerotic_glomeruli', 'non_globally_sclerotic_glomeruli']


In [6]:
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

for folder in folders:
    os.makedirs(os.path.join(train_dir, folder), exist_ok=True)
    os.makedirs(os.path.join(val_dir, folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, folder), exist_ok=True)


In [7]:
for folder in folders:
    full_path = os.path.join(img_directory, folder)
    images = os.listdir(full_path)
    
   
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)
    
   
    for image in train_images:
        shutil.copy(os.path.join(full_path, image), os.path.join(train_dir, folder, image))
        
    for image in val_images:
        shutil.copy(os.path.join(full_path, image), os.path.join(val_dir, folder, image))
        
    for image in test_images:
        shutil.copy(os.path.join(full_path, image), os.path.join(test_dir, folder, image))


In [8]:
train_dir = 'output_split/train' #path of train folder, will have subdirectories of two classes
val_dir = 'output_split/val' #path of val folder, will have subdirectories of two classes
test_dir = 'output_split/test' #path of test folder, will have subdirectories of two classes


train_datagen = ImageDataGenerator(rescale=1.0/255.0)

val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    classes={'non_globally_sclerotic_glomeruli': 0, 'globally_sclerotic_glomeruli': 1}  
)


val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes={'non_globally_sclerotic_glomeruli': 0, 'globally_sclerotic_glomeruli': 1}  
)


test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes={'non_globally_sclerotic_glomeruli': 0, 'globally_sclerotic_glomeruli': 1}  
)

Found 4606 images belonging to 2 classes.
Found 575 images belonging to 2 classes.
Found 577 images belonging to 2 classes.


In [9]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/50
144/144 [==============================] - 348s 2s/step - loss: 0.2221 - accuracy: 0.9066 - val_loss: 0.1428 - val_accuracy: 0.9357
Epoch 2/50
144/144 [==============================] - 317s 2s/step - loss: 0.1083 - accuracy: 0.9574 - val_loss: 0.1002 - val_accuracy: 0.9600
Epoch 3/50
144/144 [==============================] - 320s 2s/step - loss: 0.0735 - accuracy: 0.9742 - val_loss: 0.0775 - val_accuracy: 0.9687
Epoch 4/50
144/144 [==============================] - 340s 2s/step - loss: 0.0519 - accuracy: 0.9822 - val_loss: 0.0721 - val_accuracy: 0.9687
Epoch 5/50
144/144 [==============================] - 337s 2s/step - loss: 0.0424 - accuracy: 0.9857 - val_loss: 0.0753 - val_accuracy: 0.9687
Epoch 6/50
144/144 [==============================] - 336s 2s/step - loss: 0.0333 - accuracy: 0.9900 - val_loss: 0.1148 - val_accuracy: 0.9583
Epoch 7/50
144/144 [==============================] - 341s 2s/step - loss: 0.0271 - accuracy: 0.9917 - val_loss: 0.0920 - val_accuracy: 0.9652

In [10]:
model.save('vgg16_model.h5') #save the model weight

In [13]:
test_loss, test_acc = model.evaluate(test_generator)

predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)

y_true = test_generator.classes

print(classification_report(y_true, y_pred, target_names=['non_sclerotic', 'sclerotic']))

19/19 [==============================] - 21s 1s/step - loss: 0.0958 - accuracy: 0.9723
               precision    recall  f1-score   support

non_sclerotic       0.98      0.99      0.98       471
    sclerotic       0.95      0.90      0.92       106

     accuracy                           0.97       577
    macro avg       0.96      0.94      0.95       577
 weighted avg       0.97      0.97      0.97       577

